In [1]:
import numpy as np
import pandas as pd

## Load Dataset

In [2]:
invoice = pd.read_csv('..\\data\\raw\\invoice_wo_data_2019_20200128.csv')
calls = pd.read_pickle('..\\data\\interim\\call_history_ai_2019.pkl')

## Parse Document Numbers

In [7]:
# helper function
def find_unique(s):
    uniques = s.dropna().unique()
    if len(uniques) > 1:
        return ', '.join([item for item in uniques])
    else:
        return uniques

# preprocess invoice data
invoice['RootPDN'] = invoice.loc[:, 'PDN'].str.slice(stop=9)
invoice = (pd.pivot_table(invoice, values='TotalPartSales',
                          index='RootPDN', aggfunc=np.sum)
           .reset_index())    

# preprocess all columns containing parts document number in call history
PDN_COLS = ['PartsDocumentNumber', 'Notes']
for col in PDN_COLS:
    calls.loc[:, col] = calls.loc[:, col].str.upper()

# extract full & partial PartsDocumentNumber (PDN)
pdn_pat = '\d{2}[a-zA-Z]\d{6}'
pdn_mask = (calls.loc[:, 'PartsDocumentNumber']
            .str.match(r'{}'.format(pdn_pat))
            .fillna(False))

pdn_df = (calls.loc[pdn_mask, 'PartsDocumentNumber']
          .str.extractall(r'(?P<RootPDN>{}|\b\d+)'.format(pdn_pat)))

for idx in pdn_df.loc[pdn_df.RootPDN.apply(lambda x: len(x) < 9), :].index.values:
    partial_pdn = pdn_df.loc[idx, 'RootPDN']
    missing_length = 9 - len(partial_pdn)
    fixed_pdn = ('{}{}').format(pdn_df.loc[(idx[0], 0), 'RootPDN'][:missing_length], partial_pdn)
    pdn_df.loc[idx, 'RootPDN'] = fixed_pdn

notes_df = (calls.loc[:, 'Notes']
            .str.extractall(r'(?P<PDNinNotes>{})'.format(pdn_pat)))

full_pdn_df = pdn_df.join(notes_df, how='outer').reset_index()
full_pdn_df = (pd.melt(full_pdn_df, id_vars=['MeetingID', 'match'], value_name='RootPDN')
               .drop(columns=['match', 'variable'])
               .drop_duplicates()
               .dropna())

full_pdn_w_revenue = (full_pdn_df.merge(invoice.loc[:, ['RootPDN', 'TotalPartSales']], how='left')
                      .groupby('MeetingID')
                      .agg({'RootPDN': find_unique, 'TotalPartSales': 'sum'}))

calls = calls.join(full_pdn_w_revenue, on='MeetingID', how='left').reset_index()
calls['SalesType'] = calls.loc[:, 'RootPDN'].str.extract(r'([CS])')
calls['Branch'] = calls.loc[:, 'Branch'].str.strip()

## Export Interim & Processed Dataset

In [8]:
calls.to_pickle('..\\data\\interim\\call_history_2019_w_revenue.pkl')
calls.to_csv('..\\data\\processed\\call_history_2019_w_revenue.csv', index=False)